In [36]:
import tensorflow as tf
from keras.preprocessing.image import save_img
from keras.preprocessing.image import load_img
import os,time,sys
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""
from keras.layers import Input
from mydatabinary1 import *
from m_resunet import ResUnetPlusPlus
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import CSVLogger
import keras.backend as K
from keras.models import Model
from pathlib import Path
#from m_resunet import ResUnetPlusPlus
#from unet1 import *
from unet2 import *
#from morph_layers import *

In [37]:
import numpy as np
np.e**-0.3

0.7408182206817179

In [38]:
def opening_closing_nw(img):
  img_tensor = tf.convert_to_tensor(img, dtype=tf.float32)
  input4D = tf.cast(tf.expand_dims(img_tensor, 0), tf.float32)
  #kernel = tf.ones([5, 5, 3], tf.float32, '3d')
  kernel1 = np.array([[[0,0,0],[0,0,0],[1,1,1],[0,0,0],[0,0,0]],[[0,0,0],[1,1,1],[1,1,1],[1,1,1],[0,0,0]],[[1,1,1],[1,1,1],[1,1,1],[1,1,1],[1,1,1]],[[0,0,0],[1,1,1],[1,1,1],[1,1,1],[0,0,0]],[[0,0,0],[0,0,0],[1,1,1],[0,0,0],[0,0,0]]])
  kernel = tf.convert_to_tensor(kernel1, dtype=tf.float32)

  #Apply image opening (that is erosion followed by dilation)
  img_erosion = tf.nn.erosion2d(input4D, kernel, strides=(1,1,1,1), padding="SAME", data_format='NHWC',dilations=(1,1,1,1))
  img_dilation = tf.nn.dilation2d(img_erosion, kernel, strides=(1,1,1,1), padding="SAME", data_format='NHWC',dilations=(1,1,1,1))
  img_opening = img_dilation

  #Apply image closing (that is dilation followed by erosion)
  img_dilation = tf.nn.dilation2d(img_opening, kernel, strides=(1,1,1,1), padding="SAME", data_format='NHWC',dilations=(1,1,1,1))
  img_final = tf.nn.erosion2d(img_dilation, kernel, strides=(1,1,1,1), padding="SAME", data_format='NHWC',dilations=(1,1,1,1))
  return img_final

def recall_m(y_true, y_pred):
    y_pred = K.reshape(y_pred, shape= ((K.shape(y_pred)[0]*K.shape(y_pred)[1]*K.shape(y_pred)[2]),num_classes))
    y_true = K.reshape(y_true, shape= ((K.shape(y_true)[0]*K.shape(y_true)[1]*K.shape(y_true)[2]),num_classes))
    true_positives = K.sum(K.sum(K.round(K.clip(y_true * y_pred, 0, 1)),axis=0)[0:-1]) # Without background, which is the final class
    possible_positives = K.sum(K.sum(K.round(K.clip(y_true, 0, 1)),axis=0)[0:-1])
    recall = true_positives / (possible_positives+ K.epsilon())
    return recall
def precision_m(y_true, y_pred):
    y_pred = K.reshape(y_pred, shape= ((K.shape(y_pred)[0]*K.shape(y_pred)[1]*K.shape(y_pred)[2]),num_classes))
    y_true = K.reshape(y_true, shape= ((K.shape(y_true)[0]*K.shape(y_true)[1]*K.shape(y_true)[2]),num_classes))
    true_positives = K.sum(K.sum(K.round(K.clip(y_true * y_pred, 0, 1)),axis=0)[0:-1])
    predicted_positives = K.sum(K.sum(K.round(K.clip(y_pred, 0, 1)),axis=0)[0:-1])
    precision = true_positives / (predicted_positives+ K.epsilon())
    return precision
def fscore(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+ K.epsilon()))
def jaccard_distance_loss(y_true, y_pred, smooth=100):
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

def colour_code(image, label_values):
    x = np.argmax(image, axis = -1)
    colour_codes = np.array(label_values)
    x = colour_codes[x.astype(int)]
    return x

def fscore1(y_true,y_pred):
    y_true = y_true[:,:,:]
    y_pred = y_pred[:,:,:]
    y_true = K.flatten(y_true)
    y_pred = K.flatten(y_pred)  
    true_positives = K.sum(K.round(y_true * y_pred))
    predicted_positives = K.sum(K.round(y_pred))
    possible_positives = K.sum(K.round(y_true))
    precision = true_positives / (predicted_positives+ K.epsilon())   
    recall = true_positives / (possible_positives+ K.epsilon())  
    f_score1 = 2*((precision*recall)/(precision+recall+ K.epsilon()))
    return f_score1

def fscore(y_true,y_pred):
    y_true = y_true[:,:,:,1]
    y_pred = y_pred[:,:,:,1]
    y_true = K.round(K.flatten(y_true))
    y_pred = K.round(K.flatten(y_pred)) 
    true_positives = K.sum(y_true * y_pred)
    predicted_positives = K.sum(y_pred)
    possible_positives = K.sum(y_true)
    precision = true_positives / (predicted_positives+ K.epsilon())   
    recall = true_positives / (possible_positives+ K.epsilon())  
    f_score = 2*precision*recall/(precision+recall+ K.epsilon())
    return f_score,precision,recall

def weighted_dice_loss(weights):
  def dice_loss(y_true, y_pred):
    Kweights = K.constant(weights)
    y_true = tf.cast(y_true, tf.float32)
    #y_pred = tf.math.sigmoid(y_pred)
    numerator = 2 * tf.reduce_sum(y_true * y_pred)
    denominator = tf.reduce_sum(y_true + y_pred)
    return 1 - numerator*Kweights / denominator   
  return dice_loss
def weighted_categorical_crossentropy(weights):
    # weights = [0.9,0.05,0.04,0.01]
    def wcce(y_true, y_pred):
        Kweights = K.constant(weights)
        #if not K.is_tensor(y_pred): y_pred = K.constant(y_pred)
        y_true = K.cast(y_true, y_pred.dtype)
        return K.categorical_crossentropy(y_true, y_pred) * K.sum(y_true * Kweights, axis=-1)
    return wcce

In [39]:
import os
path = os.path.dirname(os.getcwd())
path = Path(path)
print(path)

d:\bac_clas


In [40]:

predict_folder = path/"testmodel/predtest/images"
output_folder = path/"testmodel/predtest/35pred"

weights_folder =path/"modeloutputs/weights/weights_resUnetpp_withoutweights"


loss_fn = 'categorical_crossentropy'

arch = ResUnetPlusPlus(input_size=None,no_classes=34)
model = arch.build_model()
print(model.summary())

model.load_weights("%s/weights.bac.035_resunetpp.hdf5"%weights_folder)



Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_123 (Conv2D)             (None, None, None, 1 448         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_111 (BatchN (None, None, None, 1 64          conv2d_123[0][0]                 
__________________________________________________________________________________________________
activation_69 (Activation)      (None, None, None, 1 0           batch_normalization_111[0][0]    
____________________________________________________________________________________________

In [41]:
newInput = Input(shape=(1504,2048,3))    # let us say this new InputLayer
newOutputs = model(newInput)
model1 = Model(newInput, newOutputs)

In [42]:
model1.summary()
print(predict_folder)

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 1504, 2048, 3)]   0         
_________________________________________________________________
model_6 (Functional)         (None, None, None, 34)    4070946   
Total params: 4,070,946
Trainable params: 4,063,554
Non-trainable params: 7,392
_________________________________________________________________
d:\bac_clas\testmodel\predtest\images


In [43]:
model1.layers[0]
print(str(predict_folder))

d:\bac_clas\testmodel\predtest\images


In [44]:

# t,img_name = validation_tif_dibas(predict_folder)
# # print(img.shape)
# # # t=testdatagen(str(predict_folder)[:-7])
# img = cv2.cvtColor(cv2.imread("D:\\bac_clas\\testmodel\\predtest\\images\\Actinomyces.israeli_0021.tif"),cv2.COLOR_BGR2RGB)
# # z= model1.predict((np.array([img[:1504,:,:]])/255))
# # z= model1.predict(t,batch_size=1,verbose=0,steps=None)
# z= model1.predict(img)

In [45]:

a=os.path.join(os.getcwd(),'predtest','images')
b=os.path.join(os.getcwd(),'predtest','masks')
imgs=os.listdir(a)
masks=os.listdir(b)
print(len(imgs))
k=0
for i in range(int(len(imgs))):
    print(os.path.join(b,masks[i]))

    img= cv2.cvtColor((cv2.imread(os.path.join(a,imgs[i]))),cv2.COLOR_BGR2RGB)
    msk = cv2.cvtColor(cv2.imread(os.path.join(b,masks[i])),cv2.COLOR_BGR2RGB)
    z= model1.predict((np.array([img[:1504,:,:]])/255))
    z = colour_code(z,label_values)
    cv2.imwrite('%s/%s_35pred.png'%(output_folder,imgs[i][:-4]), cv2.cvtColor(np.uint8(z[0]), cv2.COLOR_RGB2BGR))
    cv2.imwrite('%s/%s_mask.png'%(output_folder,imgs[i][:-4]),  cv2.cvtColor(np.uint8(msk[:1504,:,:]), cv2.COLOR_RGB2BGR))
    


124
d:\bac_clas\testmodel\predtest\masks\Acinetobacter.baumanii_0017_gt.tif
d:\bac_clas\testmodel\predtest\masks\Acinetobacter.baumanii_0018_gt.tif
d:\bac_clas\testmodel\predtest\masks\Acinetobacter.baumanii_0019_gt.tif
d:\bac_clas\testmodel\predtest\masks\Acinetobacter.baumanii_0020_gt.tif
d:\bac_clas\testmodel\predtest\masks\Actinomyces.israeli_0021_gt.tif
d:\bac_clas\testmodel\predtest\masks\Actinomyces.israeli_0022_gt.tif
d:\bac_clas\testmodel\predtest\masks\Actinomyces.israeli_0023_gt.tif
d:\bac_clas\testmodel\predtest\masks\Bacteroides.fragilis_0021_gt.tif
d:\bac_clas\testmodel\predtest\masks\Bacteroides.fragilis_0022_gt.tif
d:\bac_clas\testmodel\predtest\masks\Bacteroides.fragilis_0023_gt.tif
d:\bac_clas\testmodel\predtest\masks\Bifidobacterium.spp_0021_gt.tif
d:\bac_clas\testmodel\predtest\masks\Bifidobacterium.spp_0022_gt.tif
d:\bac_clas\testmodel\predtest\masks\Bifidobacterium.spp_0023_gt.tif
d:\bac_clas\testmodel\predtest\masks\Candida.albicans_0017_gt.tif
d:\bac_clas\testmo

In [46]:

# pwd = os.getcwd()
# path = os.path.join(pwd,output_folder)
# if os.path.isdir(path) is not True:
#     os.mkdir(path)


# z = colour_code(z,label_values)
# for i in range(len(z)):

#     save_img('%s/%s_24pred.png'%(output_folder,i), z[i])



In [47]:
z.shape

(1, 1504, 2048, 3)